In [1]:
import numpy as np
np.set_printoptions(suppress=True)
import pyproj

ModuleNotFoundError: No module named 'pyproj'

In [2]:
def reproject(record, semi_major, semi_minor, source_proj, dest_proj, **kwargs):
    """
    Thin wrapper around PyProj's Transform() function to transform 1 or more three-dimensional
    point from one coordinate system to another. If converting between Cartesian
    body-centered body-fixed (BCBF) coordinates and Longitude/Latitude/Altitude coordinates,
    the values input for semi-major and semi-minor axes determine whether latitudes are
    planetographic or planetocentric and determine the shape of the datum for altitudes.
    If semi_major == semi_minor, then latitudes are interpreted/created as planetocentric
    and altitudes are interpreted/created as referenced to a spherical datum.
    If semi_major != semi_minor, then latitudes are interpreted/created as planetographic
    and altitudes are interpreted/created as referenced to an ellipsoidal datum.
    Parameters
    ----------
    record : object
          Pandas series object
    semi_major : float
              Radius from the center of the body to the equater
    semi_minor : float
              Radius from the pole to the center of mass
    source_proj : str
                      Pyproj string that defines a projection space ie. 'geocent'
    dest_proj : str
                   Pyproj string that defines a project space ie. 'latlon'
    Returns
    -------
    : list
    Transformed coordinates as y, x, z
    """
    source_pyproj = pyproj.Proj(proj = source_proj, a = semi_major, b = semi_minor)
    dest_pyproj = pyproj.Proj(proj = dest_proj, a = semi_major, b = semi_minor)

    y, x, z = pyproj.transform(source_pyproj, dest_pyproj, record[0], record[1], record[2], **kwargs)

    return y, x, z

## Input Data

In [10]:
semi_major = 1100
semi_minor = 1000
altitude = 50
lon = 35
lat = 30
radius = 0

## Compute the quaternion (as x, y, z, w)

In [11]:
ground_pt = np.asarray(reproject([lon, lat, radius], semi_major, semi_minor, 'latlong', 'geocent'))
sensor_position = np.asarray(reproject([lon, lat, radius + altitude], semi_major, semi_minor, 'latlong', 'geocent'))
look_vec = ground_pt - sensor_position
look_vec = look_vec / np.linalg.norm(look_vec)
quat = np.zeros(4)
quat[:3] = np.cross(np.array([0, 0, 1]), look_vec)
quat[3] = np.dot(np.array([0, 0, 1]), look_vec)

[ 0.49673176 -0.70940648  0.         -0.5       ]
